In [6]:
%load_ext autoreload
%autoreload 2
from path_setup import setup_project_root
root = setup_project_root()

import os
import pandas as pd
# Local modules
from etl_showcase.infrastructure.utils.file_utils import (
    read_excel_sheets,
    dataframe_to_excel,
    save_large_dataframe_to_excel,
)

data_dirs_raw = os.path.join(os.getcwd(), 'data/raw')
data_dirs_preprocessed = os.path.join(os.getcwd(), 'data/processed')

# 1. 讀取資料，並僅保留必要的欄位，並確保資料乾淨
print('Start to read data')
file_path = os.path.join(os.getcwd(), data_dirs_raw, 'Shopping_Data.xlsx')
target_sheet = 'Order_Items' 
dataset = read_excel_sheets(file_path = file_path, target_sheet = target_sheet)
df = dataset[['User_ID', 'Product_ID']].drop_duplicates()

# 2. 執行自我合併 (Self-Join)
# 透過 User_ID 將資料表與自己合併，產生所有可能的產品組合
print('Start to corss combine self')
merged = pd.merge(df, df, on='User_ID', suffixes=('_A', '_B'))

# 3. 核心過濾：
# (1) 排除自己對自己 (Product_A == Product_B)
# (2) 排除重複對應 (例如 A&B 和 B&A 只留一組，利用字母順序過濾)
print('Start to drop duplicate combine')
pair_df = merged[merged['Product_ID_A'] < merged['Product_ID_B']]

# 4. 群組化計算同時購買次數 (Pair_Count)
print('Start to group and create Pair_Count column')
result = pair_df.groupby(['Product_ID_A', 'Product_ID_B']).size().reset_index(name='Pair_Count')

# 最後的 result 會回傳給 Power BI
print('Start to export data')
data_file_path_preprocessed = os.path.join(os.getcwd(), data_dirs_preprocessed, 'Pair_Statistics.xlsx')
# dataframe_to_excel(df = result, file_path = data_file_path_preprocessed, keep_index = False)
save_large_dataframe_to_excel(result, data_file_path_preprocessed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using project root: C:\Users\USER\Documents\A.change jobs\data_science_practice
Start to read data
成功讀取資料，共 549320 筆紀錄。
Start to corss combine self
Start to drop duplicate combine
Start to group and create Pair_Count column
Start to export data
DataFrame 行數 (4370109) 超過 Excel 上限，將分割成 5 個工作表。
正在寫入 'Sheet_1'，包含 1048576 行資料...
正在寫入 'Sheet_2'，包含 1048576 行資料...
正在寫入 'Sheet_3'，包含 1048576 行資料...
正在寫入 'Sheet_4'，包含 1048576 行資料...
正在寫入 'Sheet_5'，包含 175805 行資料...
所有資料已成功寫入至檔案 'C:\Users\USER\Documents\A.change jobs\data_science_practice\etl_showcase\application\data/processed\Pair_Statistics.xlsx'。
